In [ ]:
import numpy as np
import xarray as xr
import pickle
import pandas as pd
import time
import os

import datetime
from sys import getsizeof,path


import matplotlib.pyplot as plt
import matplotlib

import theano
import theano.tensor as tt
from theano.printing import pydotprint
import pymc3 as pm
theano.config.optimizer="fast_run"

In [ ]:
a = np.arange(1,1+6*3*2).reshape(6,3,2)
b = np.arange(51,51+3*4*4).reshape(3,4,4)



a = xr.DataArray(a,dims=("age","BL","sex"),coords={"sex":["m","f"],"age":[1,2,3,4,5,6],"BL":["A","B","C"]})
b = xr.DataArray(b,dims=("sex","age","week"),coords={"sex":["f","u","m"],"age":[1,2,3,6],"week":[11,13,14,18]})

ta = tt.cast(a.values,"int64")

#print(a)
#print(b)

#print(a.indexes)

#print(ta[[0,1],[1,1]].eval())

print(a.coords)
print(b.coords)

print(a.dims,b.dims)

print(a)
print(b)



In [ ]:
def IndexProperties(x):
    try:
        steps = np.diff(x)
        stepsizes = len(set(steps))
    
        return {"countable":True,"continous":stepsizes==1,"stepsize":min(steps)}
    except:
        return {"countable":False}

def IndexMap(x,common):
    return np.array([(a in common) for a in x ])

x = np.array([1,2,3,4,8,9])
print(IndexProperties(x))

print(IndexMap(np.array([0,1,2,3]),x))

In [ ]:
def IndexSumToMatch(X,iX,iC,dims,axis,sum_dir="skip"):
    """ 
        X,iX tensor and index along axis
        iC common index
        dims,axis number of dimensions and axis
        sum_dir skip, left or right """
    slice_blueprint = [slice(None) for n in range(dims)]
    slice_blueprint[axis] = 0
    
    z = tt.zeros_like(X[slice_blueprint])
    
    print(slice_blueprint)
    print(z.eval())
    
    nc,start = 0,0
    s = z
    
    for nx,ix in enumerate(iX):
        pass
    
    
    return X

In [ ]:
"""
ModelParams keeps track of coordinate-ranges for Model-internal datasets

"""


class ModelParams(object):
    def __init__(self,coords={}):
        self.coords = coords
        
        self.params = {}
        
    def AddParam(self,param):
        
        pname = param.name
        if pname not in self.params.keys():
            self.params[pname] = param
        else:
            print("Param %s already exists",pname)
            
    def __getitem(self,name):
        return self.params.get(name,None)
    
    def Overlap(self):
        pass

class ModelParam(object):
    """ Everything is a parameter in a bayesian model """
    def __init__(self,name,coords,param,is_variable=True):
        
        self.name = name
        self.coords = coords
        self.param = param
        
    def Dims(self):
        return list(self.coords.keys())
        
    def DimIndex(self,dim_name):
        return self.Dims().index(dim_name)
        
    def Overlap(self,other,sum_missing=[]):
        """ Returns overlap of both params,
            returns A overlap with B, B with A, Common coords
        """
        dims,other_dims = self.Dims(),other.Dims()
        dimS,other_dimS = set(dims),set(other_dims)
        
        dim_overlap = dimS.intersection(other_dimS)
        overlap,other_overlap = self.param, other.param
        if len(dim_overlap) > 0:
            # figure non-matching dimensions, sum over non-matching dimensions
            not_in,other_not_in = dimS-other_dimS,other_dimS-dimS
            not_index,other_not_index = list(map(self.DimIndex,not_in)),list(map(other.DimIndex,other_not_in))
            if len(not_in) > 0:
                overlap = overlap.sum(axis=not_index)
                for k in not_in:
                    dims.remove(k)
            if len(other_not_in) > 0:
                other_overlap = other_overlap.sum(axis=other_not_index)
                for k in other_not_in:
                    other_dims.remove(k) # dims, other_dims are the new indices
            
            # Transpose matching dimensions
            other_transpose = list(map(other_dims.index,dims))
            other_overlap = other_overlap.dimshuffle(other_transpose)
            
            # Reduce coordinates for both theano objects
            overlap_index = {k:self.coords[k] for k in dims}
            other_index = {other_dims[i]:other.coords[other_dims[i]] for i in other_transpose}
            
            # Do the nasty stuff
            A,B,common_index = self.Overlap_Axes(overlap,other_overlap,overlap_index,other_index,sum_missing)
            return A,B,common_index
        else:
            return None,None,None
        
    def Overlap_Axes(self,A,B,A_index,B_index,sum_missing=[]):
        """ Returns theano objects A,B
        
            For each axis, compare indizes and slice inputs A,B to match each other
        """
        
        A_indexer,B_indexer,common_index = [],[],{}
        for axis,dim in enumerate(A_index.keys()):
            iA,iB = A_index[dim], B_index[dim] 
            print(axis)
            print(iA,iB)
            print(sorted(set(iA).union(set(iB))))
            
            iC = np.array(sorted( set(iA).intersection(set(iB)) ))
            common_index[dim] = iC
            
            iPA,iPB = IndexProperties(iA),IndexProperties(iB)
            
            if iPA["countable"] == iPB["countable"] and iPA["countable"] == False:
                # Simple case for non-countable indices, only skipping with a boolean mask
                A_indexer.append(IndexMap(iA,iC))
                B_indexer.append(IndexMap(iB,iC))
                
            else: # more complex case, might involve skipping/summing of elements
                iPC = IndexProperties(iC) 
                print(iPA,iPB,iPC)
                
                if iPC["continous"] == True:
                    for name,X,iX,iP,indexer in zip(["A","B"],[A,B],[iA,iB],[iPA,iPB],[A_indexer,B_indexer]):
                        if iPC["stepsize"] == iP["stepsize"]:
                            indexer.append( slice(iX.index(iC[0]),iX.index(iC[-1])+1) )
                        else: # involves skipping/summing in 1 axis
                            IndexSumToMatch(X,iX,iC,len(A_index.keys()),axis)
                            
                            
                            indexer.append(slice(None)) # Dummy slice
                        
                else: # involves skipping/summing in A and/or B
                    print(axis,iPC)
                    
                    for name,X,iX,iP,indexer in zip(["A","B"],[A,B],[iA,iB],[iPA,iPB],[A_indexer,B_indexer]):
                        IndexSumToMatch(X,iX,iC,len(A_index.keys()),axis)
                        indexer.append(slice(None))        
        
        return A[A_indexer],B[B_indexer],common_index
    
    
    """
               # drop missing, using index-mask
            # test if indices along a dim are continous, otherwise element-wise
            slices,other_slices = [],[]
            for axis,dim in enumerate(dims):
                # Summation is done internally.
                continous = False
                
                if continous == False:
                    pass
                
                # if dim not in sum_missing:
                
                print("a=",self.coords[dim])
                print("b=",other.coords[dim])
            
                slices.append(slice(None))
                other_slices.append(slice(None))
            
            overlap = overlap[slices]
            other_overlap = other_overlap[other_slices]
            
            # sum missing
    
    """
        

class ObservedData(ModelParam):
    def __init__(self,name,data):
        """ data : xarray"""
        coords = {} # xarray.DataArray.coords is not properly ordered.
        for d in data.dims:
            coords[d] = sorted(data.coords[d].values) # Make sure indices are sorted as well.
        param = theano.shared(data.sel(coords).values)
        super(ObservedData,self).__init__(name,coords,param,is_variable=False)
        
        
    

In [ ]:
o1 = ObservedData("by_sex_BL_age",a)

o2 = ObservedData("by_sex_age_week",b)


ov1,ov2,ocoords = o1.Overlap(o2)
print("coords:",ocoords)



od1 = ov1.eval()
od2 = ov2.eval()

print(od1.shape)
print(od2.shape)

print(od1)

print(od2)


In [ ]:
def Overlap(a,b):
    """
        - trivial case: both Parameters have the same dimensions and indexes along those dimensions (coords)
        - same indexes, but different dimensions: sum over unique dimensions
        - missing indexes: either leave out (sex,BL) or try to sum in more fine grained (temporal,week)
    
    """
   
    ksa,ksb = set(a.dims),set(b.dims)
    
    sa,sb = a.sum(ksa-ksb),b.sum(ksb-ksa)
    
    print(ksa,ksb)
    print(ksa-ksb)
    print(ksb-ksa)
    
    print(sa)
    print(sb)
#    a.coords
    
Overlap(a,b)    

In [ ]:


sr,ar,br,wr = ["m","f"],range(0,11),range(1,17),range(10,30)
coords = {"sex":sr,"age":ar,"BL":br,"week":wr}

z = np.zeros(list(map(len,[sr,ar,br,wr])),dtype="int64")


print(z.shape)


for sj,s in enumerate(sr):
    si = {"m":1,"f":2}.get(s,0)
    for aj,age in enumerate(ar):
        ai = si*1000+age
        for bj,bl in enumerate(br):
            bi = ai*1000+bl
            for wj,week in enumerate(wr):
                wi = bi*10000+week
                z[sj,aj,bj,wj] = wi
                
a = xr.DataArray(z,dims=coords.keys(),coords=coords)
#print(a)     

In [ ]:
a = slice(3,9,2)
c = slice(None,3)
print(a,c)

b = tt.cast(np.arange(160).reshape(20,4,2),"int64")

d = tt.cast([True,False,True,False],"bool")
print(tt.flatten(b[a,d]).eval()[c])

In [ ]:
slice(10,20,1)+slice(30,35,2)